# SVM-gradient decent
## 1 实现方法
几乎相同于课堂上展示的伪代码，但是使用np与矩阵来进行计算，加速计算速度也减少代码量。
最终的损失函数为：
$$L(w,w_0|D)=\left\{
    \begin{aligned}
&\frac{\lambda}{2} ||w||^2 &y^{(l)}(w^Tx^{(l)}+w_0)\geq 1\\
&\frac{1}{N}\Sigma_{l=1}^N 1-y^{(l)}(w^Tx^{(l)}+w_0)+\frac{\lambda}{2} ||w||^2 & otherwise\\
\end{aligned}
\right.$$
其中，$\lambda$表示为下述PENALTY。
详细地，方法为：
1. 随机产生初始的w值。
2. 对于$w_1-w_N$，首先计算所有满足$y^{(l)}(w^Tx^{(l)}+w_0)\geq 1$对应的下标集合$I$，对于每个下标，计算$\Delta w_j=-\frac{1}{N} \cdot \Sigma_{i \in I}x_j^{(i)}\cdot y^{(i)}+PENALTY \cdot w_j$。对于$w_0$，$\Delta w_0 = -\Sigma_{i \in I}y^{(i)}$。
3. 更新$w_0=w_0-STEP\_SIZE \cdot \Delta w_0$，$w=w-STEP\_SIZE \cdot \Delta w$。
将测试数据输入模型，得到准确度。准确度的计算方式是，模型计算出的预测值与实际值差的绝对值的平均。同时由于初始随机性的选取，我将代码运行5次，准确度与时间计算取平均值。
## 2 参数
经过测试与选取，我选择了如下参数：
PENALTY = 0.001
THRESHOLD = 1e-9
STEP_SIZE = 0.0003
MAX_ITERATION = 8000
其中，选择PENALTY = 0.001是为了降低w的平方和的占比，选择THRESHOLD = 1e-9和MAX_ITERATION = 8000是为了在保证时间较短情况下增加准确度，选择STEP_SIZE = 0.0003是为了增加准确度，使得其缓慢迭代。
## 3 结果
根据上述参数配置，得到结果如下：
train_test: 0.9325
test_acc: 0.92
Running time: 2.78125 Second
测试可以发现，其在THRESHOLD约为$1 \cdot 10^{-9}$时达到稳定。同时，设置为此值时，迭代次数差不多为8000（当设置为$1 \cdot 10^{-8}$时，迭代次数约为4000）。
测试可以发现，其基本保持不变，因此我选择最大值，即STEP_SIZE = 0.0003。
对于PENALTY来说，由于并未规定取多少，因此我取了可以使得运行时间最短同时准确率最高的0.001。
此时，运行时间为2.78125s，准确率为0.92。
最终结果为：
w=[-0.61092617,-2.29493064e-01,-1.32799101e-02,-2.37979400e-01,2.50455259e-01
   -1.45104633e-01,9.34174521e-02,3.64139625e-02,-1.06806357e-01,-1.27525606e-01
   -1.87934020e-01,1.07382368e-01,-2.22253625e-01,-8.32127053e-02,-2.72055261e-01
    7.61677805e-02,-2.29022182e-02,-1.11535143e-01,3.70389924e-02,-6.91222326e-03
    7.80859202e-03,3.87350089e-02,-2.39034455e-04,-4.33990394e-03,4.08749327e-02
   -3.02514495e-02,1.03504090e-02,3.15397126e-02,-4.06747076e-03,-7.86600723e-04]

In [5]:
import time
import numpy as np

PATH_X_TRAIN = "./data/X_train.csv"
PATH_Y_TRAIN = "./data/Y_train.csv"
PATH_X_TEST = "./data/X_test.csv"
PATH_Y_TEST = "./data/Y_test.csv"
IS_CONTAINHEAD = True
IS_NORMALIZE = False
NEED_RESHAPE = True
PENALTY = 0.0001
THRESHOLD = 1e-9
STEP_SIZE = 0.0003
MAX_ITERATION = 8000

# 打印选项
# 打印每固定迭代次数结果
PRINT_EPOCH = True
PRINT_RATE = 200
# 打印总结果
PRINT_RES = True
# 打印运行时间
PRINT_TIME = True


def change_dir():
    import os
    import sys
    os.chdir(sys.path[0])
    return sys.path[0]


def read_csv(path, option="x", need_reshape=NEED_RESHAPE):
    data = np.loadtxt(path, dtype=float, delimiter=',',
                      skiprows=int(IS_CONTAINHEAD))
    if option == "y":
        data[data == 0] = -1
        if need_reshape:
            data = np.reshape(data, (np.shape(data)[0], 1))
    return data


def normalize_data(data):
    mean = np.mean(data, axis=0)
    std = np.std(data, axis=0)
    for i in range(data.shape[0]):
        data[i, :] = (data[i, :] - mean) / std
    return data


def caculate_by_gradient_decent(train_x, train_y, test_x, test_y, penalty=PENALTY, threshold=THRESHOLD, step_size=STEP_SIZE, max_iteration=MAX_ITERATION, is_normalize=IS_NORMALIZE):
    if (is_normalize):
        train_x = normalize_data(train_x)
        test_x = normalize_data(test_x)
    data_num = np.shape(train_x)[0]
    feature_dim = np.shape(train_x)[1]
    w = np.random.rand(feature_dim, 1)*0.01
    w_0 = np.random.rand(1)*0.01

    it = 1
    th = 0.1
    while it < max_iteration and th > threshold:
        a = np.tile(w_0, [data_num, 1])+train_x@w
        ksi = a*train_y
        index = (ksi < 1)[:, 0]

        dw = np.zeros([feature_dim, 1])
        dw_0 = 0
        # for w_other
        dw = -np.transpose(train_x[index])@train_y[index]
        dw = dw/data_num+penalty*w
        # for w_0
        dw_0 = -sum(train_y[index])/data_num

        w_0_ = w_0-step_size*dw_0
        w_ = w-step_size*dw

        th = np.sum(np.square(w_ - w)) + np.square(w_0_ - w_0)
        it = it + 1

        w = w_
        w_0 = w_0_

        if PRINT_EPOCH and it % PRINT_RATE == 0:
            predict_y = caculate_predict_y(w_0, w, test_x)
            print("epoch:", it, "th:", th, "acc:", caculate_acc(
                test_y, predict_y), "loss:", caculate_loss(test_y, predict_y, w_0, w))
    return w_0, w


def caculate_predict_y(w_0, w, data_x):
    predict_y = np.tile(
        w_0, [np.shape(data_x)[0], 1])+data_x@w
    predict_y = [[1] if i > 0 else [-1] for i in predict_y]
    return predict_y


def caculate_acc(test_y, predict_y):
    correct_prediction = np.equal(predict_y, test_y)
    accuracy = np.mean(correct_prediction.astype(np.float64))
    return accuracy


def caculate_loss(test_y, predict_y, w_0, w, penalty=PENALTY):
    ksi = 1-test_y*predict_y
    index = (ksi < 0)[:, 0]
    ksi[index] = 0
    loss = penalty/2*(w_0*w_0+sum(w*w))+sum(ksi)/np.shape(test_y)[0]
    return loss


def main():
    data_x_train = read_csv(PATH_X_TRAIN, "x")
    data_y_train = read_csv(PATH_Y_TRAIN, "y")
    data_x_test = read_csv(PATH_X_TEST, "x")
    data_y_test = read_csv(PATH_Y_TEST, "y")
    # print("read done!")
    res_w_0, res_w = caculate_by_gradient_decent(data_x_train,
                                                 data_y_train, data_x_test, data_y_test)
    if PRINT_RES:
        print("train_acc:", caculate_acc(data_y_train,
                                         caculate_predict_y(res_w_0, res_w, data_x_train)))
        print("test_acc:", caculate_acc(data_y_test,
                                        caculate_predict_y(res_w_0, res_w, data_x_test)))
        print(res_w_0, res_w)


if __name__ == "__main__":
    start_time = time.process_time()
    change_dir()
    main()
    end_time = time.process_time()
    if PRINT_TIME:
        print("Running time: %s Second" % (end_time-start_time))


epoch: 200 th: [0.00020503] acc: 0.605 loss: [0.79000089]
epoch: 400 th: [0.00016291] acc: 0.76 loss: [0.48000172]
epoch: 600 th: [0.00014868] acc: 0.805 loss: [0.39000256]
epoch: 800 th: [0.00010908] acc: 0.83 loss: [0.34000345]
epoch: 1000 th: [0.00010175] acc: 0.845 loss: [0.31000434]
epoch: 1200 th: [9.88169835e-05] acc: 0.835 loss: [0.33000523]
epoch: 1400 th: [0.00010148] acc: 0.855 loss: [0.2900062]
epoch: 1600 th: [0.00010414] acc: 0.855 loss: [0.29000723]
epoch: 1800 th: [0.00011213] acc: 0.87 loss: [0.2600083]
epoch: 2000 th: [0.00010143] acc: 0.875 loss: [0.25000938]
epoch: 2200 th: [0.00010346] acc: 0.875 loss: [0.2500105]
epoch: 2400 th: [1.03285445e-05] acc: 0.845 loss: [0.31001164]
epoch: 2600 th: [2.72818845e-06] acc: 0.875 loss: [0.25001276]
epoch: 2800 th: [2.11272992e-05] acc: 0.855 loss: [0.29001393]
epoch: 3000 th: [0.00011975] acc: 0.86 loss: [0.28001512]
epoch: 3200 th: [3.38460937e-05] acc: 0.87 loss: [0.26001629]
epoch: 3400 th: [1.558432e-06] acc: 0.88 loss: [